In [7]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [8]:
load_dotenv()

True

In [9]:
instruction1 = "You are a sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
    You write professiona;, serious emails."
instruction2 = "You are a humours, engaging sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.\
    You write witty,engaging cold emails that are likely to get a response."
instruction3 = "You are a busy sales agent working for ComplAI, \
    a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.\
    You write concise, to the point cold emails."

In [10]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instruction1,
    model="gpt-4o-mini"
)
sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instruction2,
    model="gpt-4o-mini"
)
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instruction1,
    model="gpt-4o-mini"
)

In [ ]:
# result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
# async for event in result.stream_events():
#     if event.type == "eaw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
#         print(event.data.delta,end = "",flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")
    print("="*30)

Subject: Elevate Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit readiness with the power of AI.

As organizations increasingly face scrutiny regarding their data security practices, ensuring seamless compliance can be a daunting challenge. ComplAI not only simplifies this process but also enhances your ability to maintain readiness for audits through automated documentation and real-time monitoring.

Here are a few key benefits our platform offers:

- **Automated Compliance Tracking**: Reduce the manual workload associated with compliance management.
- **AI-Driven Insights**: Gain actionable insights to proactively address compliance gaps.
- **Streamlined Audit Preparation**: Ensure your team is always audit-ready with minimized disruptions.

I would love the opportunity to discuss how ComplAI can specifically

In [11]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
        Imagine you are a customer and pick the one you are most likely to respond to. \
        Do not give an explination; reply with the selected email only."
)

In [11]:
message = "Write a cold sales email"

with trace("Selecting most relevent emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1,message),
        Runner.run(sales_agent2,message),
        Runner.run(sales_agent3,message)
    )
    outputs = [result.final_output for result in results]
    
    emails = "Cold sales emails: \n\n".join(outputs)
    
    best = await Runner.run(sales_picker,emails)
    
    print(f"Best sales email: \n{best.final_output}")

Best sales email: 
Subject: Don’t Let SOC2 Compliance Be a Pain in the Audit!

Hi [Recipient's Name],

Are you ready for the next round of audits, or are you thinking you need a breather (and perhaps a stiff drink)? 🤔

At ComplAI, we know that preparing for SOC2 compliance can feel like herding cats while juggling flaming swords—super stressful and not exactly a walk in the park. But what if I told you we have an AI-powered tool that can turn that chaos into a gentle stroll?

Our platform helps you simplify compliance processes, automate tedious tasks, and get you audit-ready faster than your morning coffee kicks in. ☕ It’s like having a cheat code for SOC2—minus the awkwardness of entering it in public.

Want to see for yourself? I’d love to set up a quick chat (or a virtual coffee, your pick!) to walk you through how we can make compliance feel less like a burden and more like a breeze.

Let me know when you’re free, and I promise to bring my best dad jokes along too.

Looking forwar

In [ ]:
# @function_tool
# def send_email(body: str):
#     """ Send out an email with the given body to all the sales prospects """
#     sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
#     from_email = Email('tycoon1156@gmail.com')
#     to_email = To('anandreddy.s3215@gmail.com')
#     content = Content("text/plain",body)
#     mail = Mail(from_email,to_email,"Sales email",content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

In [13]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1",tool_description="Write a cold sales email")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2",tool_description="Write a cold sales email")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3",tool_description="Write a cold sales email")

In [ ]:
# tools = [tool1,tool2,tool3,send_email]

In [ ]:
# instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
# You never generate sales emails yourself; you always use the tools. \
# You try all 3 sales_agent tools once before choosing the best one. \
# You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


# sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

# message = "Send a cold sales email addressed to 'Dear CEO'"

# with trace("Sales manager"):
#     result = await Runner.run(sales_manager, message)

In [20]:
subject_instruction = "You can write a subject for a cold email. \
    You are given a message and you need to write a subject for an email that is likely to get a response."
    
html_instruction = 'You can convert a text email body to an HTML email body. \
    You are given a text email body which n=might have some markdown \
    and you need to convert it to an HTML email body with simple, clear, compelling layout and design.'

subject_writer = Agent(name="Email subject writer",
                       instructions=subject_instruction,
                       model="gpt-4o-mini")
html_converter = Agent(name="HTML converter",
                       instructions=html_instruction,
                       model= "gpt-4o-mini")

subject_tool = subject_writer.as_tool(tool_name='subject_writer',
                                      tool_description="Write a subject for a cold sales email.")
html_tool = html_converter.as_tool(tool_name="html_converter",
                                   tool_description="Convert a text email body to an HTML email body")



In [23]:
@function_tool
def send_html_email(subject:str,html_body:str) -> Dict[str,str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRIP_API_KEY"))
    from_email = Email('tycoon1156@gmail.com')
    to_email = To('anandreddy.s3215@gmail.com')
    content = Content("text/html",html_body)
    mail = Mail(from_email,to_email,subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}

In [24]:
tools = [subject_tool,html_tool,send_html_email]

In [25]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020DE159F600>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020DE4660540>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to

In [28]:
instructions = "You are an email formattor and sender. You receive the body of an email to be sent. \
You first use the subject_tool to write the subject for the email, then use the html_converter tool to convert the bosy to an HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."
    
email_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

In [32]:
tools = [tool1,tool2,tool3]
handoffs = [email_agent]

In [35]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send out a cold sales email addressed to Dear Anand from Ajay"

In [36]:
with trace("Automated Emails"):
    result = await Runner.run(sales_manager,message)